In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import numpy as np
import allel
import itertools
import os
from subprocess import call
from tqdm import tqdm, trange
from scipy.stats import chi2_contingency

import statsmodels.api as sm
import statsmodels.formula.api as smf

#allows multiple outputs: all, last, last_expr(default), none, last_expr_or_assign
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

#### Step 1: make LD dataframe

In [ ]:
#N/A for Bombus

#### Step 2: Chop by 100k window

In [ ]:
#this step uses tabix
#this first requires turning vcf to vcf.gz
# bgzip your.vcf
# tabix -p vcf your.vcf.gz <- this makes index
# tabix your.vcf.gz chr1:10,000,000-20,000,000

#bombus has to run in hap-r2


#### Step 3: Calculating Mean-Median R2 per Window

In [ ]:
#take all files and make one dataframe
ld_r2 = []

path_folder = '/data3/TaeFile/HapLd/'

for file in tqdm(os.listdir(path_folder), total=len(path_folder)):
    window = file.split('_headered')[0]
    df = pd.read_csv(f'{path_folder}/{file}', sep='\t')
    mean_r2 = df['R^2'].mean()
    median_r2 = df['R^2'].median()
    ld_r2.append([window, mean_r2, median_r2])

#destination of the final file is in home directory = windowed_LD.csv
#condition, hap.ld, with maf 0.2, and window of 60 bp

In [ ]:
df = pd.DataFrame(ld_r2, columns = ['ID', 'r2_mean', 'r2_median'])
display(df.head())

#### Step 4: Making the Master Dataframe

In [ ]:
#AA caller, based on frequency.
#common, high frequency seen as reference/ancestral
#rarer, low frequency seen as alternative/derived
def AA_caller(frequency, reference, alternative):
    if (frequency > 0.5):
        return alternative
    elif (frequency < 0.5):
        return reference
    else:
        return np.nan

In [ ]:
#Mutation direction function "strength_classifier"
strong_bases= ['G', 'C']
weak_bases= ['A', 'T']

def strength_classifier(ancestor, derived):
    if (ancestor in strong_bases) and (derived in weak_bases):
        return 'SW'
    elif (ancestor in weak_bases) and (derived in strong_bases):
        return 'WS'
    else:
        return 'NN'

In [ ]:
#Define frequency of the Derived State, the mutation
#if ALT = Derived, keep the original AF, which describes the frequency of the ALT
#if REF = Derived, use 1 - AF

def mutation_frequency (Derived, ALT, AF):
    if Derived == ALT: #this means derived is ALT, which AF is associated with
        return AF
    if Derived != ALT: #this means dervied is REF, which is inversely associated with AF
        return (1-AF)
    else:
        return 'Error'

In [ ]:
#Ancestry based on allele frequency
#sorting based on mutation frequency

def barcoder(strength, frequency):
    if (strength == 'SW') and (frequency <= 0.1): #make it less or equal, to be generalizable for different data.
        return 'SW-Rare'
    elif (strength == 'SW') and (0.25 <= frequency <= 0.5):
        return 'SW-Common'
    elif (strength == 'WS') and (frequency <= 0.1):
        return 'WS-Rare'
    elif (strength == 'WS') and (0.25 <= frequency <= 0.5):
        return 'WS-Common'
    else:
        return 'NaN' #keep it this way for Bombus, np.nan causes error downstream, I don't know why.

In [ ]:
#execute everything
bombus_things = []

path_folder_2 = '/data3/TaeFile/HeaderedVcf/'

for file in tqdm(os.listdir(path_folder_2), total=len(path_folder_2)):
    window = file.split('_headered')[0]
    
    # Process the file
    df_basket = pd.read_table(f'{path_folder_2}/{file}', sep ='\t', header=None, comment='#')
    df_basket.rename(columns={
        0:"SCAF", 
        1:"POS", 
        2:"Id", 
        3:"REF", 
        4:"ALT", 
        5:"quality", 
        6:"filter", 
        7:"INFO", 
        8:"header", 
        9:"1", 10:"10", 11:"11", 12:"2b", 13:"3", 14:"4", 15:"5", 16:"6", 17:"7", 18:"8"}, 
                     inplace=True)
    column_picks= ["SCAF", "POS", "REF", "ALT", "INFO"]
    df_basket_picks = df_basket[column_picks]
    
    # Get Allele Frequency
    df_basket_picks['AF'] = df_basket_picks['INFO'].str.split('AF=').str.get(1).str.split(';').str.get(0).astype(float)
    df_basket_picks.drop(columns=['INFO'], inplace=True)
    
    #Drop Allele Frequency of 0 and 1
    df_basket_picks = df_basket_picks[df_basket_picks['AF'] != 1.0] #drop all AF of 1
    df_basket_picks = df_basket_picks[df_basket_picks['AF'] != 0] #drop all AF of 0
    
    #AA base calling
    df_basket_picks["AA"] = df_basket_picks.apply(lambda row: AA_caller(row["AF"], row["REF"], row["ALT"]), 
                                                  axis= 'columns')
    df_basket_picks["Derived"] = df_basket_picks.apply(lambda row: AA_caller(row["AF"], row["ALT"], row["REF"]), 
                                                       axis= 'columns')
    
    #Mutation direction
    df_basket_picks['Dirct'] = df_basket_picks.apply(lambda row: strength_classifier(row['REF'], row['ALT']), 
                                                         axis='columns')
    
    #Mutation Frequency, feed the variables in order of Derived, ALT, AF
    df_basket_picks['MF'] = df_basket_picks.apply(lambda row: mutation_frequency(row['Derived'], row['ALT'], row['AF']), 
                                                  axis='columns')
    
    df_basket_picks['Barcode'] = df_basket_picks.apply(lambda row: barcoder(row['Dirct'], row['MF']), axis='columns')
    #Barcoded_Bombus = df_basket_picks[df_basket_picks['Barcode'] != 'NaN'] #drop anything NaN <- this tosses NN
        #this also got rid of any WS and SW that fell in 0.2 and 0.4 freq window. Now it keeps it all. = better
    Barcoded_Bombus = df_basket_picks #maintain variable transition so that I don't have to touch anything downstream
    
    # dr.kent's stats
    SW_Total_freq = (Barcoded_Bombus['Dirct'].values == 'SW').sum()
    WS_Total_freq = (Barcoded_Bombus['Dirct'].values == 'WS').sum()
    NN_Total_freq = (Barcoded_Bombus['Dirct'].values == 'NN').sum()
    SNP_Total = SW_Total_freq + WS_Total_freq + NN_Total_freq
    
    SW_Rare_freq = (Barcoded_Bombus['Barcode'].values == 'SW-Rare').sum()
    WS_Rare_freq = (Barcoded_Bombus['Barcode'].values == 'WS-Rare').sum()
    
    SW_Common_freq = (Barcoded_Bombus['Barcode'].values == 'SW-Common').sum()
    WS_Common_freq = (Barcoded_Bombus['Barcode'].values == 'WS-Common').sum()
    
    bombus_things.append([window, SW_Total_freq, WS_Total_freq, NN_Total_freq, SNP_Total, SW_Rare_freq, WS_Rare_freq, 
                            SW_Common_freq, WS_Common_freq])

In [ ]:
final_file = pd.DataFrame(bombus_things)
final_file.head()

In [ ]:
len(final_file)

In [ ]:
final_file.rename(columns={
        0:"ID", 
        1:"SW_Total", 2:"WS_Total", 3:"NN_Total", 4: "SNP_Total",
        5:"SW_Rare", 6:"WS_Rare", 7:"SW_Common", 8:"WS_Common",}, inplace=True)

#W_Total_freq, WS_Total_freq, NN_Total_freq, SNP_Total, SW_Rare_freq, WS_Rare_freq, SW_Common_freq, WS_Common_freq

In [ ]:
final_file.head(5)

In [ ]:
#Start Merging R2 dataframe with the 'final file'
merged_Bombus = df.merge(final_file, on='ID')
Chopped_Bombus = merged_Bombus.dropna() #removes issue downstream
Chopped_Bombus.head(5)

In [ ]:
len(Chopped_Bombus)

In [ ]:
test = final_file[final_file['ID'] == 'NT_177880.1_900000'] #checking if it paired up correctly
test.head()

### Output

In [ ]:
#Run only once!
Chopped_Bombus.to_csv('/home/taeyoon/VcfFiles/LdByWindow/BombusSFiles/Chopped_Bombus.csv', index=False)

In [ ]:
Bombus_df = pd.read_csv('/home/taeyoon/VcfFiles/LdByWindow/BombusSFiles/Chopped_Bombus.csv')
Bombus_df.head(8)

#### Merge with GC content per Window and Adjust Total Value

In [ ]:
Bombus_GC = pd.read_csv('/home/taeyoon/GCContent/BimpGC_ready.csv', sep='\t')
Bombus_GC.head()

In [ ]:
#merge!
Bombus_GC_df = pd.merge(Bombus_df, Bombus_GC, how='left', on=['ID'])
Bombus_GC_df.head()
len(Bombus_GC_df)

In [ ]:
#Adjusted Total values
Bombus_GC_df['SW_T_Adjusted'] = Bombus_GC_df['SW_Total']/Bombus_GC_df['GC_Content']
Bombus_GC_df['WS_T_Adjusted'] = Bombus_GC_df['WS_Total']/(1 - Bombus_GC_df['GC_Content'])

In [ ]:
#Lambda, which is SW/WS
Bombus_GC_df['Lambda'] = Bombus_GC_df['SW_T_Adjusted'] / Bombus_GC_df['WS_T_Adjusted']

In [ ]:
Bombus_GC_df.head()

In [ ]:
len(Bombus_GC_df)

In [ ]:
#concise, easier viewing
Bombus_view = Bombus_GC_df.drop(columns=[
    'SW_Total','WS_Total','NN_Total','SNP_Total','SW_Rare','WS_Rare','SW_Common','WS_Common'])
Bombus_view.head()

In [ ]:
#mean of lambda
Bombus_view['Lambda'].mean()

In [ ]:
#GC content unadjusted
(Bombus_GC_df['SW_Total']/Bombus_GC_df['WS_Total']).mean()

### S1: Looking at Total numbers

In [ ]:
#name the the final table to work with x, drop possible NaNs
x = Bombus_GC_df.dropna()
len(x)

In [ ]:
#plot WS and SW together
plt.figure(figsize=(12,6))
plt.ylim(0,2100)
plt.xlim(0,1)

#WS is blue
WST_adj = sns.regplot(x['r2_mean'], x['WS_T_Adjusted'], marker="+", scatter_kws={'alpha':0.5}, label='WS') 

#SW is orange
sns.regplot(x['r2_mean'], x['SW_T_Adjusted'], marker="+", scatter_kws={'alpha':0.5}, label='SW') 

plt.ylabel('GC% adjusted Mutation Counts')
plt.xlabel('R^2 Mean')
plt.legend(loc='upper right', prop={'size': 20}, markerscale=2)

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['WS_T_Adjusted'])#WS is blue

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['SW_T_Adjusted']) #SW is orange

In [ ]:
#Z-test for coefficients (slopes)
def Z_score(slope1, std_error1, slope2, std_error2):
    numerator = (slope1 - slope2)
    denominator = pow((pow(std_error1,2) + pow(std_error2,2)), 1/2)
    Z = (numerator) / (denominator)
    return Z

In [ ]:
#Z_score, input order in slope1, std_error1, slope2, std_error2
Z_score(-556.3163997919412, 22.419131687548198, -1479.556418050194, 61.61696031931891)
#Z score: 14.08047801884966
#p-value: 5.007e-45, reject null

#### Unadjusted

In [ ]:
#unadjusted values
#plot WS and SW together
plt.figure(figsize=(12,6))
plt.ylim(0,1000)
plt.xlim(0,1)

#WS is blue
WST_adj = sns.regplot(x['r2_mean'], x['WS_Total'], marker="+", scatter_kws={'alpha':0.5}, label='WS') 

#SW is orange
sns.regplot(x['r2_mean'], x['SW_Total'], marker="+", scatter_kws={'alpha':0.5}, label='SW') 

plt.ylabel('Mutation Counts')
plt.xlabel('R^2 Mean')
plt.legend(loc='upper right', prop={'size': 20}, markerscale=2)

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['WS_Total']) #WS is blue

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['SW_Total']) #SW is orange

In [ ]:
#Z_score, input order in slope1, std_error1, slope2, std_error2
Z_score(-333.0772246646324, 14.575893956648743, -596.2502791390713, 21.889007231972794)
#Z score: 10.00733728308455
#p-value: 1.415e-23, reject null

### Lambda

In [ ]:
#SW/WS, adjusted
plt.figure(figsize=(12,6))
plt.ylim(0,6)
plt.xlim(0,1)

WST_adj = sns.regplot(x['r2_mean'], x['Lambda'], marker="+", scatter_kws={'alpha':0.5}, label='Lambda') 
scipy.stats.linregress(x['r2_mean'], x['Lambda']) 

plt.ylabel('Lambda')
plt.xlabel('R^2 Mean')
plt.legend(loc='upper right', prop={'size': 20}, markerscale=2)

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['Lambda']) 

#### Unadjusted Lambda

In [ ]:
#unadjusted SW/WS
plt.figure(figsize=(12,6))
plt.ylim(0,3)
plt.xlim(0,1)

WST_adj = sns.regplot(x['r2_mean'], x['SW_Total']/x['WS_Total'], marker="+", 
                      scatter_kws={'alpha':0.5}, label='Lambda, raw') 

plt.ylabel('Lambda')
plt.xlabel('R^2 Mean')
plt.legend(loc='upper right', prop={'size': 20}, markerscale=2)

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['SW_Total']/x['WS_Total']) 

### S2: The 10%, Rares

In [ ]:
#Plot rare occuring mutations both direction (SW and WS)
plt.figure(figsize=(12,6))
plt.ylim(0,500)
plt.xlim(0,1)

#WS is blue
WST_adj = sns.regplot(x['r2_mean'], x['WS_Rare'], marker="+", scatter_kws={'alpha':0.5}, label='WS') 

#SW is orange
sns.regplot(x['r2_mean'], x['SW_Rare'], marker="+", scatter_kws={'alpha':0.5}, label='SW') 

plt.ylabel('Rare Mutation Counts')
plt.xlabel('R^2 Mean')
plt.legend(loc='upper right', prop={'size': 20}, markerscale=2)

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['WS_Rare']) #WS is blue

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['SW_Rare']) #SW is orange

In [ ]:
#Z_score, input order in slope1, std_error1, slope2, std_error2
Z_score(-137.42182589971887, 5.97001400336962, -331.3243591219772, 11.574309307219929)
#Z score: 14.88892207971511
#p-value: 3.89e-50, reject null

#### Adjusted (note, order reversed)

In [ ]:
#adjusted for GC%
plt.figure(figsize=(12,6))
plt.ylim(0,1200)
plt.xlim(0,1)

#WS is blue
WST_adj = sns.regplot(x['r2_mean'], x['WS_Rare']/(1 - x['GC_Content']), marker="+", scatter_kws={'alpha':0.5}, label='WS') 

#SW is orange
sns.regplot(x['r2_mean'], x['SW_Rare']/x['GC_Content'], marker="+", scatter_kws={'alpha':0.5}, label='SW') 

plt.ylabel('GC% Adjusted Rare Mutation Counts')
plt.xlabel('R^2 Mean')
plt.legend(loc='upper right', prop={'size': 20}, markerscale=2)

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['WS_Rare']/(1 - x['GC_Content'])) #WS is blue

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['SW_Rare']/x['GC_Content']) #SW is orange

In [ ]:
#Z_score, input order in slope1, std_error1, slope2, std_error2
Z_score(-231.09078524314677, 9.244594424933686, -816.0914951965949, 31.308788830152398)
#Z score: 17.920013931749107
#p-value: 8.231e-72, two-tailed, reject null

### S3: The 50%, Common

In [ ]:
#Common, both directions
plt.figure(figsize=(12,6))
plt.ylim(0,200)
plt.xlim(0,1)

#WS is blue
WST_adj = sns.regplot(x['r2_mean'], x['WS_Common'], marker="+", scatter_kws={'alpha':0.5}, label='WS') 

#SW is orange
sns.regplot(x['r2_mean'], x['SW_Common'], marker="+", scatter_kws={'alpha':0.5}, label='SW') 

plt.ylabel('Common Mutation Counts')
plt.xlabel('R^2 Mean')
plt.legend(loc='upper right', prop={'size': 20}, markerscale=2)

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['WS_Common']) #WS is blue

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['SW_Common']) #SW is orange

In [ ]:
#Z_score, input order in slope1, std_error1, slope2, std_error2
Z_score(-124.50003857900936, 5.803275326954651, -149.01341361218874, 6.617117051168298)
#Z score: 2.7851748838878825
#p-value: 0.00535, reject null

#### Adjusted for GC content (note reversed order)

In [ ]:
#adjusted for GC%
plt.figure(figsize=(12,6))
plt.ylim(0,600)
plt.xlim(0,1)

#WS is blue
WST_adj = sns.regplot(x['r2_mean'], x['WS_Common']/(1 - x['GC_Content']), marker="+", scatter_kws={'alpha':0.5}, label='WS') 

#SW is orange
sns.regplot(x['r2_mean'], x['SW_Common']/x['GC_Content'], marker="+", scatter_kws={'alpha':0.5}, label='SW') 

plt.ylabel('GC% Adjusted Common Mutation Counts')
plt.xlabel('R^2 Mean')
plt.legend(loc='upper right', prop={'size': 20}, markerscale=2)

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['WS_Common']/(1 - x['GC_Content'])) #WS is blue

In [ ]:
scipy.stats.linregress(x['r2_mean'], x['SW_Common']/x['GC_Content']) #SW is orange

In [ ]:
#Z_score, input order in slope1, std_error1, slope2, std_error2
Z_score(-206.97054727948316, 8.95909717632094, -373.4737218168506, 19.07827699119179)
#Z score: 7.899702175042606
#p-value: 2.796e-15, reject null

### Two window comparison, Odds Ratio

In [ ]:
#selecting for datapoints within the window or r2_mean 0.35~0.45 and 0.7~0.9
bombus_slice_1 = x[x['r2_mean'].between(0.35, 0.45, inclusive=True)];
bombus_slice_2 = x[x['r2_mean'].between(0.7,0.9, inclusive=True)];

In [ ]:
bombus_slice_1.head()

In [ ]:
bombus_slice_2.head()

In [ ]:
#categorizing data for Odds ratio, no adjustments for GC%, as it would make no difference.
#first for first window, denoted slice 1
sliced1_WS_R_adj = bombus_slice_1['WS_Rare'].sum();
sliced1_WS_C_adj = bombus_slice_1['WS_Common'].sum();

sliced1_SW_R_adj = bombus_slice_1['SW_Rare'].sum();
sliced1_SW_C_adj = bombus_slice_1['SW_Common'].sum();

In [ ]:
#OR slice 1, WS/SW, which with Common/Rare is:
(sliced1_WS_C_adj/sliced1_WS_R_adj)/(sliced1_SW_C_adj/sliced1_SW_R_adj)
#1.8812483529947406

In [ ]:
#Chi square, slice 1
chi_slice1 = np.array ([[sliced1_SW_R_adj, sliced1_SW_C_adj], [sliced1_WS_R_adj, sliced1_WS_C_adj]]) 
#array setup: SW-Rare,Common and WS-Rare,Common
chi2_contingency(chi_slice1)
#this returns chi-sqaure, p, degrees of freedom, and expected values in array
#(3318.7735798257568, 0.0, 1, array([[63361.04007827, 34481.95992173],[35928.95992173, 19553.04007827]]))

In [ ]:
#categorizing data for Odds ratio, adjusted for GC%
#second window, denoted slice 2
sliced2_WS_R_adj = bombus_slice_2['WS_Rare'].sum();
sliced2_WS_C_adj = bombus_slice_2['WS_Common'].sum();

sliced2_SW_R_adj = bombus_slice_2['SW_Rare'].sum();
sliced2_SW_C_adj = bombus_slice_2['SW_Common'].sum();

In [ ]:
#OR slice 2, WS/SW is:
(sliced2_WS_C_adj/sliced2_WS_R_adj)/(sliced2_SW_C_adj/sliced2_SW_R_adj)
#1.7314384541973955

In [ ]:
#Chi square, slice 2
chi_slice2 = np.array ([[sliced2_SW_R_adj, sliced2_SW_C_adj], [sliced2_WS_R_adj, sliced2_WS_C_adj]]) 
#array setup: SW-Rare,Common and WS-Rare,Common
chi2_contingency(chi_slice2)
#this returns chi-sqaure, p, degrees of freedom, and expected values in array
#(1672.9979163350395, 0.0, 1, array([[42443.68350748, 22129.31649252],[25117.31649252, 13095.68350748]]))